In [1]:
# !pip install torchsummary

# 과제

<span style = 'font-size:1.3em;line-height:1.5em'><b>1. </b>실습파일 '3_Transfer_Learning.ipynb'에서 활용한 데이터로 Resnet18모델에 대해서 transfer learning을 수행해보세요. 단, 동일한 실습파일에서 수행한 방식처럼 마지막 fc layer에 대해서만 fine-tuning하는 방식으로 수행해보세요.</span>

In [1]:
import os, time, copy
import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import torchvision

from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.optim import lr_scheduler
from torchvision import datasets, models, transforms
from torchvision.datasets.folder import ImageFolder
from torchvision.models import resnet18

import matplotlib.pyplot as plt
from torchsummary import summary
%matplotlib inline

In [ ]:
# 데이터셋의 변환 정의
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

# 데이터셋 로딩
dataset_path = 'data/cats_and_dogs/PetImages/'  

dataset = ImageFolder(root=dataset_path, transform=data_transforms)

# 트레이닝과 검증 인덱스 분할
def create_splits(dataset, train_size=0.8):
    num_train = int(len(dataset) * train_size)
    indices = list(range(len(dataset)))
    np.random.shuffle(indices)
    train_indices, val_indices = indices[:num_train], indices[num_train:]
    return train_indices, val_indices

train_indices, val_indices = create_splits(dataset)

# DataLoader 설정
train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=32, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=32, sampler=val_sampler)

# ResNet18 모델 로딩 및 수정
model = models.resnet18(pretrained=True)
for param in model.parameters():
    param.requires_grad = False  # 기존 파라미터는 고정

# 마지막 fc 레이어 교체
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)  # 클래스 수를 2로 설정

# 모델을 사용할 device 설정 (CUDA가 사용 가능하면 GPU 사용)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)  # fc 레이어의 파라미터만 최적화

# 학습 함수 정의
def train_model(model, criterion, optimizer, num_epochs=25):
    for epoch in range(num_epochs):
        model.train()  # 모델을 트레이닝 모드로 설정
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # 에포크별 손실률 출력
        epoch_loss = running_loss / len(train_loader.dataset)
        print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}')
        
        # 여기에 검증 과정을 추가할 수 있습니다.
    return model

# 모델 학습
model_trained = train_model(model, criterion, optimizer, num_epochs=25)


C:\Users\SANDBOX\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\SANDBOX\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
C:\Users\SANDBOX\anaconda3\Lib\site-packages\PIL\TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Epoch 0/24, Loss: 0.0030
